<a href="https://colab.research.google.com/github/Serendipity-Song/Classify_CIFAR-10_Dataset/blob/main/Cifar_10%2C_ResNet_110_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

# Normalize pixel values between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convert class vectors to binary class matrices
train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)



In [ ]:

# Define a ResNet block
def resnet_block(input, filters, kernel_size=3, stride=1, conv_shortcut=True):
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding="same",
                      kernel_initializer='he_normal')(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters, kernel_size, padding="same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)

    if conv_shortcut:
        input = layers.Conv2D(filters, 1, strides=stride, kernel_initializer='he_normal')(input)
        input = layers.BatchNormalization()(input)

    x = layers.Add()([x, input])
    x = layers.Activation('relu')(x)
    return x



In [ ]:

# Build ResNet-110 model
def build_resnet110(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(16, 3, activation='relu', kernel_initializer='he_normal', padding="same")(inputs)
    x = layers.BatchNormalization()(x)

    # First stack
    for _ in range(18):
        x = resnet_block(x, 16)

    # Second stack
    x = resnet_block(x, 32, stride=2, conv_shortcut=True)
    for _ in range(17):
        x = resnet_block(x, 32)

    # Third stack
    x = resnet_block(x, 64, stride=2, conv_shortcut=True)
    for _ in range(17):
        x = resnet_block(x, 64)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)



In [ ]:

# Compile and train the model
input_shape = train_images.shape[1:]
num_classes = 10

model = build_resnet110(input_shape, num_classes)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, batch_size=128,
                    validation_data=(test_images, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'\nTest accuracy: {test_acc}')


In [ ]:
# Save the model
model.save('')